In [1]:
import intake
from intake import open_catalog
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

In [2]:
load_dotenv()
db_user_vulcain = os.getenv("DB_USER_VULCAIN")
os.getenv("DB_PWD_VULCAIN")
os.getenv("DB_HOST_VULCAIN")
os.getenv("DB_PORT_VULCAIN")
os.getenv("DB_VULCAIN_PROD")

usr=os.getenv("DB_USER")
pswd=os.getenv("DB_PWD")
host=os.getenv("DB_HOST")
port=os.getenv("DB_PORT")
db_traitement=os.getenv("DB_WORKSPACE")
db_ref=os.getenv("DB_REF")
db_externe=os.getenv("DB_EXT")
dwh_fact_strategy=os.getenv("DWH_FACT_STRATEGY")
dwh_dim_strategy=os.getenv("DWH_DIM_STRATEGY")
home=os.getenv("HOME_PATH")
data_catalog_dir=os.getenv("DATA_CATALOG_DIR")

In [3]:
def getEngine(user=usr,pswd=pswd,host=host,dbase=db_traitement):
    connection = f'postgresql://{user}:{pswd}@{host}:{port}/{dbase}'
    return create_engine(connection)

In [4]:
cat_data_georep = open_catalog(f'{data_catalog_dir}data_externe_georep.yaml')
epsg=3163
schema='dimenc'
db = db_externe
engine = getEngine(dbase=db)

In [6]:
cat_data_georep.concessions_minieres.name

'concessions_minieres'

In [9]:
l = ['concessions_minieres']
for i in l:
    data= getattr(cat_data_georep,i).read().to_crs(epsg)
    data.to_postgis(i,engine, schema=schema,if_exists='replace')
    print(i," to pgis")
    with engine.connect() as con:
        con.execute(f'ALTER TABLE {i} ADD PRIMARY KEY (`objectid`);')

concessions_minieres  to pgis


ObjectNotExecutableError: Not an executable object: 'ALTER TABLE concessions_minieres ADD PRIMARY KEY (`objectid`);'

In [ ]:
postgresString = """postgresql://{{env("DB_USER")}}:{{env("DB_PWD")}}@{{env("DB_HOST")}}:{{env("DB_PORT")}}/{{env("DB_EXT")}}"""
for i in l:
    
    with open(f'{data_catalog_dir}data_reference.yaml', 'a') as f:
        f.write(f"""  
  {i}:
    args:
      geopandas_kwargs:
        geom_col: geometry
        crs: 3163
      uri: {postgresString}
      sql_expr: select * from {schema}.{i}
      table : {schema}.{i}
    description: source de données {i}
    driver: intake_geopandas.geopandas.PostGISSource
        """)

NameError: name 'l' is not defined